<a href="https://colab.research.google.com/github/eispoohw/CS493-Math-Methods-in-ML/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [2]:
!gdown 1PRxLQ1YvXRylaTF8ks1a5BhGLFyoCA61
!gdown 1HDVBsj7p0AePBXs_S9qweDGU0s0htTVQ
!gdown 1cgGkHek94tauBpbtMN2CCQC0ACTOzQkv
!gdown 1olXMkeQuv4UV--b0bQFusqR-zf0keP8S

Downloading...
From: https://drive.google.com/uc?id=1PRxLQ1YvXRylaTF8ks1a5BhGLFyoCA61
To: /content/gender_train.csv
100% 99.9k/99.9k [00:00<00:00, 45.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HDVBsj7p0AePBXs_S9qweDGU0s0htTVQ
To: /content/tr_mcc_codes.csv
100% 14.9k/14.9k [00:00<00:00, 21.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1cgGkHek94tauBpbtMN2CCQC0ACTOzQkv
To: /content/tr_types.csv
100% 14.2k/14.2k [00:00<00:00, 19.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1olXMkeQuv4UV--b0bQFusqR-zf0keP8S
To: /content/transactions.csv
100% 314M/314M [00:04<00:00, 67.3MB/s]


In [3]:
gender_train = pd.read_csv('gender_train.csv', sep=',')
tr_mcc_codes = pd.read_csv('tr_mcc_codes.csv', sep=';', index_col='mcc_code')
tr_types = pd.read_csv('tr_types.csv', sep=';', index_col='tr_type')
transactions = pd.read_csv('transactions.csv', sep=',', nrows=1000000)

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [15]:
full_sample = transactions.sample(1000)['tr_type']
full_sample.to_frame()

,tr_type
17503,7070
774071,1010
111168,1010
370304,1030
951264,1010
...,...
507795,7030
421950,7070
255057,1010
177082,2370


In [16]:
valid_sample = full_sample.loc[full_sample.isin(tr_types.index)]
satisfy = tr_types.loc[valid_sample]['tr_description'].str.contains('POS|АТМ', regex=True).sum()

res = satisfy / len(full_sample)

print(f"Sample length: {len(full_sample)}\n\
Valid indexes: {len(valid_sample)}\n\
Satisfying values: {satisfy}\n\
Result: {res}")

Sample length: 1000
Valid indexes: 998
Satisfying values: 606
Result: 0.606


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [17]:
freq = transactions['tr_type'].value_counts(normalize=True, ascending=False)
freq.to_frame()

,tr_type
1010,0.231117
2010,0.151166
7070,0.149006
1110,0.137658
1030,0.118975
...,...
2446,0.000004
4096,0.000004
8146,0.000003
1510,0.000002


In [18]:
largest_freq = freq.nlargest(10)
freq_desc = tr_types.loc[largest_freq.index]
freq_desc['tr_frequency'] = round(largest_freq, 3)
freq_desc

,tr_description,tr_frequency
1010,Покупка. POS ТУ СБ РФ,0.231
2010,Выдача наличных в АТМ Сбербанк России,0.151
7070,Перевод на карту (с карты) через Мобильный бан...,0.149
1110,Покупка. POS ТУ Россия,0.138
1030,Оплата услуги. Банкоматы СБ РФ,0.119
2370,Списание с карты на карту по операции <перевод...,0.050
7010,Взнос наличных через АТМ (в своем тер.банке),0.029
7030,Перевод на карту (с карты) через АТМ (в предел...,0.026
7071,Перевод на карту (с карты) через Мобильный бан...,0.016
1100,Покупка. ТУ Россия,0.015


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [19]:
cust_money = transactions[['customer_id','amount']]

In [20]:
income = cust_money.where(cust_money.amount > 0).groupby(by='customer_id').sum('amount').sort_values(by='amount', ascending=False)
income.head(1)

,amount
customer_id,
70780820.0,1.248115e+09


In [21]:
outcome = cust_money.where(cust_money.amount < 0).groupby(by='customer_id').sum('amount').sort_values(by='amount', ascending=True)
outcome.head(1)

,amount
customer_id,
70780820.0,-1.249952e+09


In [45]:
idxes = list(set([income.index[0], outcome.index[0]]))
res = pd.DataFrame({
    'difference': abs(income.amount.loc[idxes] + outcome.amount.loc[idxes])
})
res

,difference
customer_id,
70780820.0,1837317.98


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [46]:
indexes = largest_freq.index.sort_values()
appropriate = transactions[transactions.tr_type.isin(indexes)][['tr_type', 'amount']]

pd.DataFrame({
    'mean': appropriate.groupby('tr_type').mean('amount')['amount'],
    'median': appropriate.groupby('tr_type').median('amount')['amount']
})

,mean,median
tr_type,,
1010,-19784.748640,-7411.52
1030,-5320.980222,-2245.92
1100,-44061.827262,-10188.26
1110,-32119.330371,-11207.57
2010,-136077.629325,-44918.32
2370,-205418.249032,-44918.32
7010,276391.789596,112295.79
7030,86104.332909,13951.52
7070,65569.831700,11319.42


In [47]:
indexes = [outcome.head(1).index, income.head(1).index]
appropriate = transactions[transactions.customer_id.isin(indexes)][['customer_id', 'amount']]

pd.DataFrame({
    'mean': appropriate.groupby('customer_id').mean('amount')['amount'],
    'median': appropriate.groupby('customer_id').median('amount')['amount']
})

,mean,median
customer_id,,
70780820,-20.694946,8803.99


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [48]:
nt = transactions.merge(gender_train, how='left', on='customer_id')
nt = nt.merge(tr_types, how='inner', on='tr_type')
nt = nt.merge(tr_mcc_codes, how='inner', on='mcc_code')
nt

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,tr_description,mcc_description
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,Оплата услуги. Банкоматы СБ РФ,"Звонки с использованием телефонов, считывающих..."
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,Оплата услуги. Банкоматы СБ РФ,"Звонки с использованием телефонов, считывающих..."
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,Оплата услуги. Банкоматы СБ РФ,"Звонки с использованием телефонов, считывающих..."
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,Оплата услуги. Банкоматы СБ РФ,"Звонки с использованием телефонов, считывающих..."
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,Оплата услуги. Банкоматы СБ РФ,"Звонки с использованием телефонов, считывающих..."
...,...,...,...,...,...,...,...,...,...
999579,48819028,388 00:00:00,7993,1200,-8062.84,00000000,0.0,Покупка. Зарубеж. ТУ,Принадлежности для видеоигр
999580,32478197,254 00:00:00,7993,1200,-5367.74,00000181,0.0,Покупка. Зарубеж. ТУ,Принадлежности для видеоигр
999581,32478197,254 00:00:00,7993,1200,-5816.92,00000181,0.0,Покупка. Зарубеж. ТУ,Принадлежности для видеоигр
999582,89619970,21 00:00:00,7993,1200,-67355.01,NaN,NaN,Покупка. Зарубеж. ТУ,Принадлежности для видеоигр


## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [49]:
outcome_by_gender = nt[nt.amount < 0][['gender','amount']].groupby('gender').mean('amount')
round(abs(outcome_by_gender.iloc[0, 0] - outcome_by_gender.iloc[1, 0]), 2)

32718.05

In [51]:
income_by_gender = nt[nt.amount > 0][['gender','amount']].groupby('gender').mean('amount')
round(abs(income_by_gender.iloc[0, 0] - income_by_gender.iloc[1, 0]), 2)

63366.57

## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [52]:
incomes = nt[nt.amount > 0].groupby(['gender', 'tr_type']).max('amount').rename(columns={'amount': 'max_income'})
smallest = incomes.groupby(level=0, group_keys=False).apply(lambda x: x.nsmallest(10, ['max_income']))
smallest

customer_id  mcc_code  max_income
gender tr_type                                   
0.0    4051        65368200      4829     1122.96
       4110        14808847      6011     2245.92
       4210        67193294      6011     2245.92
       2370        44552122      4829     2335.75
       4100        34267056      6011     5041.86
       7075        14169381      6010     6737.75
       2110        14808847      6011    22459.16
       1010        69189450      7832    24839.83
       2010        69189450      6011    44918.32
       2210        67193294      6011    68315.82
1.0    2020        46579237      6010     2245.92
       4110        99048480      6011     2245.92
       1010        40877619      5411    28803.87
       8100        47653609      5813    64682.37
       6000        61024099      3000    79280.83
       7041         6839194      6010    87590.72
       2010        91582559      6011    89836.63
       2110        99048480      6011   112295.79
       2370        56053006      4829   113194.15
       7015        66245004      6011   121279.45

In [53]:
smallest[smallest.index.get_level_values(1).duplicated()].reset_index()[['tr_type']]

,tr_type
0,4110
1,1010
2,2010
3,2110
4,2370


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [54]:
outcomes = nt[nt.amount < 0]
outcomes = outcomes.groupby(['gender', 'mcc_code'], as_index=True).sum('amount')
outcomes

customer_id  tr_type       amount
gender mcc_code                                   
0.0    742         850440942    15540   -386207.68
       1711         24784802     1110   -674897.69
       1799        315460267     5550   -261626.73
       2741         23652302     1210    -20746.42
       3000       6997821749   174240 -74431645.27
...                      ...      ...          ...
1.0    8999      40722216455   968900 -28643346.63
       9222        876024177    17620   -988205.17
       9311        759021396    18700   -373271.22
       9399       4203808637    88900  -1810820.67
       9402         13025490     1110     -4581.67

[349 rows x 3 columns]

In [55]:
outcomes_fem = outcomes[outcomes.index.get_level_values(0) == 0].reset_index(level=0, drop=True)[['amount']]
outcomes_male = outcomes[outcomes.index.get_level_values(0) == 1].reset_index(level=0, drop=True)[['amount']]

res = outcomes_fem.merge(outcomes_male, how='inner', on='mcc_code')
res['difference'] = abs(res.amount_x - res.amount_y)
res[['difference']].nlargest(10, 'difference').merge(tr_mcc_codes, how='inner', on='mcc_code')

,difference,mcc_description
mcc_code,,
6011,3.085862e+09,Финансовые институты — снятие наличности автом...
4829,2.499153e+09,Денежные переводы
6010,2.208580e+08,Финансовые институты — снятие наличности вручную
5541,1.554326e+08,Станции техобслуживания
5511,7.352877e+07,"Легковой и грузовой транспорт — продажа, серви..."
5812,7.037856e+07,"Места общественного питания, рестораны"
5977,4.241762e+07,Магазины косметики
5533,4.233808e+07,Автозапчасти и аксессуары
7995,4.079564e+07,Транзакции по азартным играм


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

In [58]:
with_hour = nt
with_hour['tr_hour'] = [x[-1][:2] for x in with_hour.tr_datetime.str.split(' ')]
with_hour.sample(10)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,tr_description,mcc_description,tr_hour
573677,83517007,168 15:49:44,5411,1010,-7793.33,NaN,NaN,Покупка. POS ТУ СБ РФ,"Бакалейные магазины, супермаркеты",15
507199,40197046,415 16:01:55,5912,1110,-9298.09,00415137,NaN,Покупка. POS ТУ Россия,Аптеки,16
261492,97777635,0 16:51:29,6011,2010,-20213.24,NaN,NaN,Выдача наличных в АТМ Сбербанк России,Финансовые институты — снятие наличности автом...,16
144744,78772251,222 12:40:51,6011,7010,628856.42,979643,0.0,Взнос наличных через АТМ (в своем тер.банке),Финансовые институты — снятие наличности автом...,12
798952,70780820,110 10:10:17,6010,7070,3705.76,NaN,1.0,Перевод на карту (с карты) через Мобильный бан...,Финансовые институты — снятие наличности вручную,10
18842,37156682,59 10:40:39,4814,1030,-1122.96,NaN,NaN,Оплата услуги. Банкоматы СБ РФ,"Звонки с использованием телефонов, считывающих...",10
322715,69705190,81 00:33:43,4829,2330,-224591.58,NaN,NaN,Списание с карты по операции “перевода с карты...,Денежные переводы,00
65664,61853422,91 19:33:40,4814,1030,-2245.92,NaN,NaN,Оплата услуги. Банкоматы СБ РФ,"Звонки с использованием телефонов, считывающих...",19
875592,9099643,205 05:51:58,6010,7070,44918.32,NaN,NaN,Перевод на карту (с карты) через Мобильный бан...,Финансовые институты — снятие наличности вручную,05
734191,45911713,399 11:45:07,5977,1010,-3368.87,223292,0.0,Покупка. POS ТУ СБ РФ,Магазины косметики,11


In [57]:
with_hour[(with_hour.amount < 0) & (with_hour.tr_hour <= '06')].groupby('gender').agg({'amount': 'count'})

,amount
gender,
0.0,43510
1.0,46694
